# 0) chargement de spark

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

# 1) lire de la donnée

In [46]:
path_csv  = "./../data/cyclistes/"
voyages = spark.read.format("csv").load(path_csv, header=True)
voyages.count()

42000

# 2) enregistrer la table dans l'espace des noms SQL

In [47]:
nom_table = "voyages"
voyages.registerTempTable(nom_table)

# 3) exécuter une requête SQL sur ce nom de table 

In [48]:
requete_sql = "select count(*) from {nom_table}".format(nom_table=nom_table)
print(requete_sql, " :")
spark.sql(requete_sql).collect()

select count(*) from voyages  :


[Row(count(1)=42000)]

In [49]:
voyages.printSchema()

root
 |-- id: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- sur_velo: string (nullable = true)
 |-- velo: string (nullable = true)
 |-- vitesse: string (nullable = true)
 |-- position: string (nullable = true)
 |-- destination_finale: string (nullable = true)



# 4) faire une jointure

## 4.1) on enregistre une seconde table dans l'espace des noms SQL

In [52]:
path_csv  = "./../data/Villes/ville_1.csv"
cyclistes = spark.read.format("csv").load(path_csv, header=True)
cyclistes .printSchema()

root
 |-- id: string (nullable = true)
 |-- vitesse_a_pied: string (nullable = true)
 |-- vitesse_a_velo: string (nullable = true)
 |-- home: string (nullable = true)
 |-- travail: string (nullable = true)
 |-- sportif: string (nullable = true)
 |-- casseur: string (nullable = true)
 |-- statut: string (nullable = true)
 |-- salaire: string (nullable = true)
 |-- sexe: string (nullable = true)
 |-- age: string (nullable = true)
 |-- sportivite: string (nullable = true)
 |-- velo_perf_minimale: string (nullable = true)



In [53]:
cyclistes.registerTempTable("cyclistes")

## 4.2) on fait une jointure sur le champ id

In [64]:
cyclistes.printSchema()

root
 |-- id: string (nullable = true)
 |-- vitesse_a_pied: string (nullable = true)
 |-- vitesse_a_velo: string (nullable = true)
 |-- home: string (nullable = true)
 |-- travail: string (nullable = true)
 |-- sportif: string (nullable = true)
 |-- casseur: string (nullable = true)
 |-- statut: string (nullable = true)
 |-- salaire: string (nullable = true)
 |-- sexe: string (nullable = true)
 |-- age: string (nullable = true)
 |-- sportivite: string (nullable = true)
 |-- velo_perf_minimale: string (nullable = true)



In [65]:
voyages.printSchema()

root
 |-- id: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- sur_velo: string (nullable = true)
 |-- velo: string (nullable = true)
 |-- vitesse: string (nullable = true)
 |-- position: string (nullable = true)
 |-- destination_finale: string (nullable = true)



In [75]:
requete_sql = """
SELECT c.id, max(c.age), mean(v.vitesse)
FROM   cyclistes as c , voyages as v
WHERE c.id == v.id
GROUP BY c.id
"""
spark.sql(requete_sql).take(5)

[Row(id='51', max(age)='46', avg(CAST(vitesse AS DOUBLE))=0.09036380956279483),
 Row(id='7', max(age)='57', avg(CAST(vitesse AS DOUBLE))=0.35489345221780044),
 Row(id='15', max(age)='25', avg(CAST(vitesse AS DOUBLE))=0.14527881551079416),
 Row(id='54', max(age)='48', avg(CAST(vitesse AS DOUBLE))=0.1324084664679445),
 Row(id='101', max(age)='39', avg(CAST(vitesse AS DOUBLE))=0.8831228007871614)]

## 4.3) utilisation de fonctions SQL et renommage de champs

In [83]:
requete_sql = """
SELECT        upper( c.id          ) as cycliste        , 
                max( c.age         ) as age             , 
         round(mean( v.vitesse),2  ) as vitesse_moyenne ,
               cast( now() as date ) as date_actuelle
FROM     cyclistes       as c , 
         voyages         as v
WHERE    c.id == v.id
GROUP BY c.id
"""
spark.sql(requete_sql).take(5)

[Row(cycliste='51', age='46', vitesse_moyenne=0.09, date_actuelle=datetime.date(2019, 3, 29)),
 Row(cycliste='7', age='57', vitesse_moyenne=0.35, date_actuelle=datetime.date(2019, 3, 29)),
 Row(cycliste='15', age='25', vitesse_moyenne=0.15, date_actuelle=datetime.date(2019, 3, 29)),
 Row(cycliste='54', age='48', vitesse_moyenne=0.13, date_actuelle=datetime.date(2019, 3, 29)),
 Row(cycliste='101', age='39', vitesse_moyenne=0.88, date_actuelle=datetime.date(2019, 3, 29))]

# 5) utilisation de l'API des dataFrames

In [92]:
path_csv  = "./../data/Villes/ville_1.csv"
cyclistes = spark.read.format("csv").load(path_csv, header=True, inferSchema=True)
cyclistes .printSchema()

root
 |-- id: integer (nullable = true)
 |-- vitesse_a_pied: double (nullable = true)
 |-- vitesse_a_velo: double (nullable = true)
 |-- home: string (nullable = true)
 |-- travail: string (nullable = true)
 |-- sportif: boolean (nullable = true)
 |-- casseur: boolean (nullable = true)
 |-- statut: string (nullable = true)
 |-- salaire: double (nullable = true)
 |-- sexe: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- sportivite: double (nullable = true)
 |-- velo_perf_minimale: double (nullable = true)



In [93]:
cyclistes.select("id").show(5)

+---+
| id|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
+---+
only showing top 5 rows



In [97]:
cyclistes.select(["id", "age"]).show(5)

+---+---+
| id|age|
+---+---+
|  1| 15|
|  2| 49|
|  3| 71|
|  4| 69|
|  5| 38|
+---+---+
only showing top 5 rows



In [124]:
requete_sql = """
SELECT        c.sexe  ,
        count(c.id   )
FROM     cyclistes as c 
WHERE    c.age > 18
GROUP BY c.sexe
"""
spark.sql(requete_sql).show()

+----+---------+
|sexe|count(id)|
+----+---------+
|   F|       57|
|   H|       37|
+----+---------+



In [117]:
cyclistes.select(["id",  "sexe"]).where(cyclistes["age"] > 18).groupby("sexe").count().show()

+----+-----+
|sexe|count|
+----+-----+
|   F|   57|
|   H|   37|
+----+-----+



In [102]:
adultes = cyclistes.filter(cyclistes["age"] > 18)
mineurs = cyclistes.filter(cyclistes["age"] < 18)
mineurs.count(), adultes.count()

(8, 94)